In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

/Users/hyunwonch/PycharmProjects/gs_crawln/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [22]:
def get_url(url):
    try:
        driver.get(url)
    except:
        driver.close()

In [28]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)

search_command = "Kuan-Yu Chen umich"
stop_year = 2015

print(f"Name : {search_command}, Until : {stop_year}")
url = "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C23&q=" + search_command + "&btnG="

override_url = "https://scholar.google.com/citations?user=6tBFbCQAAAAJ&hl=en"
override = 0
end_signal = 0

Name : Kuan-Yu Chen umich, Until : 2015


In [29]:
if(override):
    get_url(override_url)
    driver.find_element(By.XPATH, '//*[@id="gsc_a_ha"]').click()
    time.sleep(5)
else:
    get_url(url)
    
    # Go to actual link that contains list
    time.sleep(5)
    try:
        driver.find_element(By.XPATH, '//*[@id="gs_res_ccl_mid"]/div[1]/table/tbody/tr/td[2]/h4/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="gsc_a_ha"]').click()
        time.sleep(5)
    except:
        try:
            driver.find_element(By.XPATH, '//*[@id="gs_res_ccl_mid"]/div[1]/table/tbody/tr/td[2]/div[1]/h4/a').click()
            time.sleep(2)
            driver.find_element(By.XPATH, '//*[@id="gsc_a_ha"]').click()
            time.sleep(5)
        except:
            driver.close()
            end_signal = 1

In [31]:
#Show more button
while(1):
    if(driver.find_element(By.XPATH, '//*[@id="gsc_bpf_more"]').get_attribute('disabled')):
        break
    driver.find_element(By.XPATH, '//*[@id="gsc_bpf_more"]').click()
    time.sleep(1)
print("Show Everything")

# Get all paper links

paper_link_list = []
num = 1
while(1):
    link = '#gsc_a_b > tr:nth-child(' + str(num) + ') > td.gsc_a_t > a'
    try:
        paper_link = driver.find_elements(By.CSS_SELECTOR, link)[0].get_attribute("href")
    except:
        break
    num += 1
    paper_link_list.append(paper_link)

len(paper_link_list)
#print(paper_link_list[0])

# Save the information

title_list = []
authors_list = []
date_list = []
journal_list = []



for i in paper_link_list:
    driver.get(i)
    try:
        title = driver.find_element(By.CLASS_NAME, 'gsc_oci_title_link').text
        authors = driver.find_elements(By.CLASS_NAME, 'gsc_oci_value')[0].text
        date = driver.find_elements(By.CLASS_NAME, 'gsc_oci_value')[1].text
        journal = driver.find_elements(By.CLASS_NAME, 'gsc_oci_value')[2].text
        #print(title)
        #print(authors)
        #print(date)
        #print(journal)
        if(date.split('/')[0] == stop_year):
            break
        title_list.append(title)
        authors_list.append(authors)
        date_list.append(date)
        journal_list.append(journal)
        time.sleep(5)
    except:
        continue
driver.close()

new_authors_list = []
for i in authors_list:
    tmp = i.split(",")
    tmp_list = []
    for j in tmp:
        tmp_list.append("[[" + j.strip() + "]]")
    new_authors_list.append(tmp_list)
#print(new_authors_list[0])

print("Total Length : ",len(title_list))


file_name = "./paper_list/" + search_command + ".txt"
print(file_name)
file = open(file_name,'w')

for i in range(len(title_list)):
    if(journal_list[i] == 'US'):
        continue
    print(date_list[i], file=file)
    print("[[", title_list[i], "]]", sep='', file=file)
    for index, j in enumerate(new_authors_list[i]):
        if(index == (len(new_authors_list[i])-1)):
            print(j,"", sep='', end='\n', file=file)
        else:
            print(j,", ", sep='', end='', file=file)
    print(journal_list[i], file=file)
    print('-------------------------------------------------------------------------------------------------------------------------', file=file)

file.close()

Show Everything
Total Length :  15
./paper_list/Kuan-Yu Chen umich.txt
